In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import t
from sklearn.linear_model import LinearRegression

Đọc dữ liệu

In [ ]:
PATH = '/content/drive/MyDrive/TUDTK/SEATTLE TACOMA AIRPORT, WA US.csv'
df = pd.read_csv(PATH)
df = df.drop(['STATION', 'NAME', 'DATE'], axis=1)
df

,AWND,CDSD,CLDD,DP01,DP10,DSND,DSNW,DT00,DT32,DX32,...,WDF1,WDF2,WDF5,WDFG,WDFM,WSF1,WSF2,WSF5,WSFG,WSFM
0,NaN,0,0,16,13,0.0,0.0,0,18,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0,0,16,11,0.0,1.0,0,14,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0,0,14,7,0.0,0.0,0,8,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0,0,20,11,0.0,0.0,0,7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0,0,16,11,0.0,0.0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,8.5,362,0,21,11,0.0,0.0,0,0,0,...,NaN,210.0,180.0,NaN,NaN,NaN,33.1,50.0,NaN,NaN
886,9.4,362,0,25,20,0.0,0.0,0,1,0,...,NaN,200.0,190.0,NaN,NaN,NaN,44.1,58.0,NaN,NaN
887,9.8,362,0,22,17,5.0,3.0,0,7,5,...,NaN,200.0,190.0,NaN,NaN,NaN,30.0,44.0,NaN,NaN
888,6.9,0,0,15,12,2.0,0.0,0,7,0,...,NaN,220.0,210.0,NaN,NaN,NaN,30.0,46.0,NaN,NaN


Làm sạch dữ liệu: Loại đi các hàng có tất cả giá trị đều là NaN, sau đó loại đi các cột có ít nhất một giá trị là NaN

In [ ]:
df = df.dropna(axis = 0, how = 'all')
df = df.dropna(axis = 1, how = 'any')
df

,CDSD,CLDD,DP01,DP10,DT00,DT32,DX32,DX70,DX90,EMNT,EMXP,EMXT,HTDD,PRCP,TAVG,TMAX,TMIN
0,0,0,16,13,0,18,0,0,0,25,0.74,53,836,4.49,38.0,43.6,32.5
1,0,0,16,11,0,14,0,0,0,20,0.98,57,757,5.26,38.9,45.2,32.6
2,0,0,14,7,0,8,0,1,0,28,2.04,70,685,3.68,42.9,51.1,34.7
3,0,0,20,11,0,7,0,0,0,30,0.70,66,586,3.76,45.4,53.6,37.3
4,0,0,16,11,0,0,0,4,0,35,0.88,80,376,4.77,52.9,61.4,44.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,362,0,21,11,0,0,0,0,0,36,1.99,69,409,5.76,51.8,57.9,45.8
886,362,0,25,20,0,1,0,0,0,31,1.26,63,514,10.26,47.9,53.0,42.7
887,362,0,22,17,0,7,5,0,0,17,0.48,56,835,4.08,38.0,41.9,34.2
888,0,0,15,12,0,7,0,0,0,22,2.00,56,744,7.07,41.0,46.2,35.8


In [ ]:
# Trích xuất bảng gồm các thuộc tính không phải là thuộc tính dự đoán (PRCP) làm X_train 
#            bảng gồm các giá trị của thuộc tính dự đoán (PRCP) làm y_train 
df_Xtrain = df.drop('PRCP', axis = 1) 
df_ytrain = df['PRCP'] 

# Lấy danh sách tên các thuộc tính 
features_name = ['CONST'] + list(df_Xtrain.columns)
y_train = np.asarray(df_ytrain, dtype=float)
X_train = sm.add_constant(np.asarray(df_Xtrain, dtype=float))

Hàm hồi quy tuyến tính tổng thể: $y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_px_p + e$ \\
Hàm hồi quy tuyến tính mẫu: $\hat{y} = b_0 + b_1x_1 + b_2x_2 + ... + b_px_p$ \\
Đoạn code sau đây để tính hàm hồi quy tuyến tính mẫu bằng công thức Normal Equation


In [ ]:
model = LinearRegression(fit_intercept = False)
model.fit(X_train, y_train)

LinearRegression(fit_intercept=False)

Kiểm định t-test với mỗi thuộc tính thứ $i$ với giả thuyết: \\
$H_0: \beta_{i} = 0$ \\
$H_1: \beta_{i} \neq 0$


Tính các giá trị $t_{stat}$ (giá trị thống kê kiểm định) và $p_{value}$ để tiến hành việc kiểm định.

In [ ]:
N = len(X_train) # Số bộ 
p = len(X_train[0]) # Số thuộc tính

y_hat = model.predict(X_train) # Tính các giá trị 
RSS = np.linalg.norm(y_train - y_hat) ** 2 # Tính tổng bình phương thặng dư 
sigma_squared_hat = RSS / (N - p) # Ước lượng phương sai của sai số kết quả 

# Tính độ lệch chuẩn
var_beta_hat = np.linalg.inv(X_train.T @ X_train) * sigma_squared_hat 
standard_error = [var_beta_hat[i, i] ** 0.5 for i in range(p)] 

t_stat = np.divide(model.coef_, standard_error) # Tính giá trị thống kê kiểm định 
p_value = (1 - t.cdf(abs(t_stat), N - p)) * 2 # Tính p_value

df_stat = pd.DataFrame({'Feature': features_name, 'Std_Err': standard_error, 'T_stat': t_stat, 'p_value': p_value})
df_stat

,Feature,Std_Err,T_stat,p_value
0,CONST,2.949585,-0.128356,0.897896
1,CDSD,0.000278,3.059750,0.002283
2,CLDD,0.002454,-0.180750,0.856606
3,DP01,0.010939,3.195094,0.001448
4,DP10,0.012171,25.286365,0.000000
5,DT00,0.885751,2.635586,0.008548
6,DT32,0.011123,-1.309895,0.190576
7,DX32,0.034372,-0.976912,0.328883
8,DX70,0.009430,3.708106,0.000222
9,DX90,0.047130,1.337732,0.181332


Với độ tin cậy là 95%, ta thấy: \\
- Các thuộc tính (trừ CONST) nếu có $p_{value} > 0.05$ ta không đủ cơ sở để bác bỏ $H_0$ tức đối với các thuộc tính này đóng góp cho việc dự đoán thuộc tính đầu ra là không đáng kể nên ta sẽ loại bỏ các thuộc tính này rồi tiến hành xây dựng lại công thức hồi quy tuyến tính. 
- Các thuộc tính còn lại (trừ CONST) có $p_{value} \le 0.05$ ta bác bỏ $H_0$ nên các thuộc tính này sẽ được giữ lại để xây dựng lại công thức hồi quy tuyến tính.

In [ ]:
features_name = np.asarray(features_name[1:]) # Lấy các thuộc tính trừ CONST
selected_features = features_name[np.where(p_value[1:] <= 0.05)] # Lấy các thuộc tính có p_value <= 0.05 
print(selected_features)

['CDSD' 'DP01' 'DP10' 'DT00' 'DX70' 'EMXP']
